In [446]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

pd.set_option('display.max_columns', None)

In [447]:
players = pd.read_csv('C:/Users/parks/OneDrive/바탕 화면/DATASET/Football/players.csv')
clubs = pd.read_csv('C:/Users/parks/OneDrive/바탕 화면/DATASET/Football/clubs.csv')
valuations = pd.read_csv('C:/Users/parks/OneDrive/바탕 화면/DATASET/Football/player_valuations.csv')
games = pd.read_csv('C:/Users/parks/OneDrive/바탕 화면/DATASET/Football/games.csv')
lineup = pd.read_csv('C:/Users/parks/OneDrive/바탕 화면/DATASET/Football/game_lineups.csv')
club_games = pd.read_csv('C:/Users/parks/OneDrive/바탕 화면/DATASET/Football/club_games.csv')
game_events = pd.read_csv('C:/Users/parks/OneDrive/바탕 화면/DATASET/Football/game_events.csv')
appearances = pd.read_csv('C:/Users/parks/OneDrive/바탕 화면/DATASET/Football/appearances.csv')

In [448]:
# Players 테이블에서 필요없는 칼럼 제거
players = players.drop(['last_season', 'current_club_id', 'image_url', 'url', 'first_name', 'last_name', 'agent_name', 'city_of_birth', 'contract_expiration_date', 'market_value_in_eur'], axis=1)

In [449]:
#valuations에서 필요없는 칼럼 제거
valuations = valuations.drop(['player_club_domestic_competition_id', 'current_club_id'], axis=1)

In [450]:
#valuations date 이름 새로 지정
valuations = valuations.rename(columns={'date' : 'valuations_date'})

In [451]:
valuations = valuations.sort_values(['player_id', 'valuations_date'])

In [452]:
# player_id가 같은 행들에 대해서 valuations_date와 date를 비교하여 조건을 만족하는 경우를 찾는다.
merged_df = pd.merge(appearances, valuations, on='player_id', suffixes=('_appearances', '_valuations'))
condition = merged_df['date'] < merged_df['valuations_date']

# 조건을 만족하는 행들에 대해서 최소값을 구한다.
min_values = merged_df[condition].groupby('player_id').agg({
    'date': 'min',
    'market_value_in_eur': 'min'
}).reset_index()

# 최소값을 valuation 테이블에 추가한다.
valuations = pd.concat([valuations, min_values.rename(columns={'date': 'valuations_date', 'market_value_in_eur': 'market_value_in_eur'})])

In [453]:
# players와 valuations를 player_id로 merge
df_players = pd.merge(players, valuations, on='player_id')

In [454]:
df_players = df_players.drop(['player_code','foot', 'height_in_cm', 'highest_market_value_in_eur'], axis=1)

In [455]:
players_value_df = df_players.sort_values(['player_id', 'valuations_date'])

In [456]:
players_value_df.groupby('player_id')['valuations_date'].shift(-1)

0         2005-01-07
1         2005-05-05
2         2005-09-30
3         2006-01-09
4         2006-07-15
             ...    
490425           NaN
490427           NaN
490428           NaN
490429    2024-03-15
490430           NaN
Name: valuations_date, Length: 490431, dtype: object

In [457]:
players_value_df['lead_date'] = players_value_df.groupby('player_id')['valuations_date'].shift(-1)

In [458]:
players_value_df

,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,sub_position,position,current_club_domestic_competition_id,current_club_name,valuations_date,market_value_in_eur,lead_date
0,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2004-10-04,7000000,2005-01-07
1,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2005-01-07,9000000,2005-05-05
2,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2005-05-05,12000000,2005-09-30
3,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2005-09-30,15000000,2006-01-09
4,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2006-01-09,20000000,2006-07-15
...,...,...,...,...,...,...,...,...,...,...,...,...
490425,1201294,Ozan Demirbağ,Türkiye,Türkiye,2008-02-12,Left Winger,Attack,TR1,Adana Demirspor Kulübü,2024-03-15,75000,NaN
490427,1201580,Yusuf Yılmaz,Türkiye,Türkiye,2006-04-29,Goalkeeper,Goalkeeper,TR1,İstanbul Başakşehir Futbol Kulübü,2024-03-15,50000,NaN
490428,1214946,Sid Ahmed Aissaoui,Algeria,Algeria,2005-01-11,Central Midfield,Midfield,RU1,PFK CSKA Moskva,2024-02-02,150000,NaN
490429,1225269,Erdem Çalık,Türkiye,Türkiye,2006-06-05,Central Midfield,Midfield,TR1,Pendikspor,2024-01-28,50000,2024-03-15


In [459]:
players_value_df[players_value_df['lead_date'].isnull()]

,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,sub_position,position,current_club_domestic_competition_id,current_club_name,valuations_date,market_value_in_eur,lead_date
22,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2016-01-04,1000000,NaN
50,26,Roman Weidenfeller,Germany,Germany,1980-08-06,Goalkeeper,Goalkeeper,L1,Borussia Dortmund,2017-12-28,750000,NaN
79,65,Dimitar Berbatov,Bulgaria,Bulgaria,1981-01-30,Centre-Forward,Attack,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,2016-06-21,1000000,NaN
102,77,Lúcio,Brazil,Brazil,1978-05-08,Centre-Back,Defender,IT1,Juventus Football Club,2016-11-15,200000,NaN
134,80,Tom Starke,East Germany (GDR),Germany,1981-03-18,Goalkeeper,Goalkeeper,L1,FC Bayern München,2018-06-05,100000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
490422,1196380,Berhan Deniz,Türkiye,Türkiye,2004-11-25,Centre-Forward,Attack,TR1,Samsunspor,2024-03-15,75000,NaN
490425,1201294,Ozan Demirbağ,Türkiye,Türkiye,2008-02-12,Left Winger,Attack,TR1,Adana Demirspor Kulübü,2024-03-15,75000,NaN
490427,1201580,Yusuf Yılmaz,Türkiye,Türkiye,2006-04-29,Goalkeeper,Goalkeeper,TR1,İstanbul Başakşehir Futbol Kulübü,2024-03-15,50000,NaN
490428,1214946,Sid Ahmed Aissaoui,Algeria,Algeria,2005-01-11,Central Midfield,Midfield,RU1,PFK CSKA Moskva,2024-02-02,150000,NaN


In [460]:
players_value_df = players_value_df.fillna('2024-04-15')

In [461]:
players_value_df

,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,sub_position,position,current_club_domestic_competition_id,current_club_name,valuations_date,market_value_in_eur,lead_date
0,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2004-10-04,7000000,2005-01-07
1,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2005-01-07,9000000,2005-05-05
2,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2005-05-05,12000000,2005-09-30
3,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2005-09-30,15000000,2006-01-09
4,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2006-01-09,20000000,2006-07-15
...,...,...,...,...,...,...,...,...,...,...,...,...
490425,1201294,Ozan Demirbağ,Türkiye,Türkiye,2008-02-12,Left Winger,Attack,TR1,Adana Demirspor Kulübü,2024-03-15,75000,2024-04-15
490427,1201580,Yusuf Yılmaz,Türkiye,Türkiye,2006-04-29,Goalkeeper,Goalkeeper,TR1,İstanbul Başakşehir Futbol Kulübü,2024-03-15,50000,2024-04-15
490428,1214946,Sid Ahmed Aissaoui,Algeria,Algeria,2005-01-11,Central Midfield,Midfield,RU1,PFK CSKA Moskva,2024-02-02,150000,2024-04-15
490429,1225269,Erdem Çalık,Türkiye,Türkiye,2006-06-05,Central Midfield,Midfield,TR1,Pendikspor,2024-01-28,50000,2024-03-15


In [462]:
players_value_df.isnull().sum()

player_id                               0
name                                    0
country_of_birth                        0
country_of_citizenship                  0
date_of_birth                           0
sub_position                            0
position                                0
current_club_domestic_competition_id    0
current_club_name                       0
valuations_date                         0
market_value_in_eur                     0
lead_date                               0
dtype: int64

In [463]:
players_value_df['valuations_date'] = pd.to_datetime(players_value_df['valuations_date'])

In [464]:
players_value_df['lead_date'] = pd.to_datetime(players_value_df['lead_date'])

In [465]:
players_value_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 490431 entries, 0 to 490430
Data columns (total 12 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   player_id                             490431 non-null  int64         
 1   name                                  490431 non-null  object        
 2   country_of_birth                      490431 non-null  object        
 3   country_of_citizenship                490431 non-null  object        
 4   date_of_birth                         490431 non-null  object        
 5   sub_position                          490431 non-null  object        
 6   position                              490431 non-null  object        
 7   current_club_domestic_competition_id  490431 non-null  object        
 8   current_club_name                     490431 non-null  object        
 9   valuations_date                       490431 non-null  datetime6

In [466]:
players_value_df

,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,sub_position,position,current_club_domestic_competition_id,current_club_name,valuations_date,market_value_in_eur,lead_date
0,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2004-10-04,7000000,2005-01-07
1,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2005-01-07,9000000,2005-05-05
2,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2005-05-05,12000000,2005-09-30
3,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2005-09-30,15000000,2006-01-09
4,10,Miroslav Klose,Poland,Germany,1978-06-09,Centre-Forward,Attack,IT1,Società Sportiva Lazio S.p.A.,2006-01-09,20000000,2006-07-15
...,...,...,...,...,...,...,...,...,...,...,...,...
490425,1201294,Ozan Demirbağ,Türkiye,Türkiye,2008-02-12,Left Winger,Attack,TR1,Adana Demirspor Kulübü,2024-03-15,75000,2024-04-15
490427,1201580,Yusuf Yılmaz,Türkiye,Türkiye,2006-04-29,Goalkeeper,Goalkeeper,TR1,İstanbul Başakşehir Futbol Kulübü,2024-03-15,50000,2024-04-15
490428,1214946,Sid Ahmed Aissaoui,Algeria,Algeria,2005-01-11,Central Midfield,Midfield,RU1,PFK CSKA Moskva,2024-02-02,150000,2024-04-15
490429,1225269,Erdem Çalık,Türkiye,Türkiye,2006-06-05,Central Midfield,Midfield,TR1,Pendikspor,2024-01-28,50000,2024-03-15


In [467]:
appearances = appearances.drop(['appearance_id', 'player_name', 'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played'], axis=1)

In [468]:
appearances = appearances.rename(columns={'player_current_club_id' : 'current_club_id'})

In [469]:
df = pd.merge(players_value_df, appearances, on='player_id', how='inner')

In [470]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40552339 entries, 0 to 40552338
Data columns (total 17 columns):
 #   Column                                Dtype         
---  ------                                -----         
 0   player_id                             int64         
 1   name                                  object        
 2   country_of_birth                      object        
 3   country_of_citizenship                object        
 4   date_of_birth                         object        
 5   sub_position                          object        
 6   position                              object        
 7   current_club_domestic_competition_id  object        
 8   current_club_name                     object        
 9   valuations_date                       datetime64[ns]
 10  market_value_in_eur                   int64         
 11  lead_date                             datetime64[ns]
 12  game_id                               int64         
 13  player_clu

In [471]:
#경기 출전 기간 2018-08-01부터
df = df[df['date'] >='2018-08-01']

In [472]:
#출전 당시 프리미어리그 경기를 뛴 선수만 조회
df = df[df['competition_id'] =='GB1']

In [473]:
df = df.sort_values('date')

In [474]:
# 조건을 충족하는 행의 인덱스 가져오기
indices_to_drop = df[df['valuations_date'] > df['date']].index

# 인덱스를 사용하여 행 삭제
df.drop(indices_to_drop, inplace=True)

In [475]:
df.sort_values('player_id')

,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,sub_position,position,current_club_domestic_competition_id,current_club_name,valuations_date,market_value_in_eur,lead_date,game_id,player_club_id,current_club_id,date,competition_id
158315,2865,Stephan Lichtsteiner,Switzerland,Switzerland,1984-01-16,Right-Back,Defender,L1,FC Augsburg 1907,2011-01-10,8000000,2011-06-21,3050167,11,167,2018-08-12,GB1
159623,2865,Stephan Lichtsteiner,Switzerland,Switzerland,1984-01-16,Right-Back,Defender,L1,FC Augsburg 1907,2013-01-15,16500000,2013-06-19,3050247,11,167,2018-10-22,GB1
155975,2865,Stephan Lichtsteiner,Switzerland,Switzerland,1984-01-16,Right-Back,Defender,L1,FC Augsburg 1907,2004-12-23,1400000,2006-01-27,3050317,11,167,2018-12-08,GB1
161717,2865,Stephan Lichtsteiner,Switzerland,Switzerland,1984-01-16,Right-Back,Defender,L1,FC Augsburg 1907,2016-06-01,6000000,2016-07-15,3050317,11,167,2018-12-08,GB1
162755,2865,Stephan Lichtsteiner,Switzerland,Switzerland,1984-01-16,Right-Back,Defender,L1,FC Augsburg 1907,2017-12-29,2000000,2018-06-07,3050247,11,167,2018-10-22,GB1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40547076,1005649,Murillo,Brazil,Brazil,2002-07-04,Centre-Back,Defender,GB1,Nottingham Forest Football Club,2023-04-07,500000,2023-07-25,4095183,703,703,2023-11-05,GB1
40547077,1005649,Murillo,Brazil,Brazil,2002-07-04,Centre-Back,Defender,GB1,Nottingham Forest Football Club,2023-04-07,500000,2023-07-25,4095194,703,703,2023-11-12,GB1
40551005,1082850,Deivid Washington,Brazil,Brazil,2005-06-05,Centre-Forward,Attack,GB1,Chelsea Football Club,2023-10-28,9000000,2023-12-19,4095170,631,631,2023-10-28,GB1
40551001,1082850,Deivid Washington,Brazil,Brazil,2005-06-05,Centre-Forward,Attack,GB1,Chelsea Football Club,2023-07-25,4000000,2023-10-09,4095170,631,631,2023-10-28,GB1


In [476]:
df2 = df[['game_id', 'player_id', 'name', 'date', 'player_club_id', 'valuations_date', 'lead_date', 'market_value_in_eur']]
df2.sort_values(['game_id', 'player_id', 'valuations_date'])

,game_id,player_id,name,date,player_club_id,valuations_date,lead_date,market_value_in_eur
155705,3050167,2865,Stephan Lichtsteiner,2018-08-12,11,2004-10-04,2004-12-23,1250000
155966,3050167,2865,Stephan Lichtsteiner,2018-08-12,11,2004-12-23,2006-01-27,1400000
156227,3050167,2865,Stephan Lichtsteiner,2018-08-12,11,2006-01-27,2006-04-04,2000000
156488,3050167,2865,Stephan Lichtsteiner,2018-08-12,11,2006-04-04,2007-12-27,3000000
156749,3050167,2865,Stephan Lichtsteiner,2018-08-12,11,2007-12-27,2008-05-05,4000000
...,...,...,...,...,...,...,...,...
40258094,4095368,649317,Jhon Durán,2024-03-17,405,2023-02-04,2023-03-16,12000000
40258125,4095368,649317,Jhon Durán,2024-03-17,405,2023-03-16,2023-06-20,12000000
40258156,4095368,649317,Jhon Durán,2024-03-17,405,2023-06-20,2023-10-09,12000000
40258187,4095368,649317,Jhon Durán,2024-03-17,405,2023-10-09,2023-12-19,17000000


In [477]:
df2['date'] = pd.to_datetime(df2['date'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1216652 entries, 11772978 to 38298293
Data columns (total 8 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   game_id              1216652 non-null  int64         
 1   player_id            1216652 non-null  int64         
 2   name                 1216652 non-null  object        
 3   date                 1216652 non-null  datetime64[ns]
 4   player_club_id       1216652 non-null  int64         
 5   valuations_date      1216652 non-null  datetime64[ns]
 6   lead_date            1216652 non-null  datetime64[ns]
 7   market_value_in_eur  1216652 non-null  int64         
dtypes: datetime64[ns](3), int64(4), object(1)
memory usage: 83.5+ MB


C:\Users\parks\AppData\Local\Temp\ipykernel_22980\1119539335.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['date'] = pd.to_datetime(df2['date'])


In [478]:
df3 = df2[df2['lead_date'] >= '2018-01-01']
df3.sort_values(['game_id', 'player_id', 'valuations_date'])

,game_id,player_id,name,date,player_club_id,valuations_date,lead_date,market_value_in_eur
162752,3050167,2865,Stephan Lichtsteiner,2018-08-12,11,2017-12-29,2018-06-07,2000000
163013,3050167,2865,Stephan Lichtsteiner,2018-08-12,11,2018-06-07,2018-12-19,1500000
782249,3050167,5658,Petr Cech,2018-08-12,11,2017-06-28,2018-01-02,7000000
782503,3050167,5658,Petr Cech,2018-08-12,11,2018-01-02,2018-05-28,6000000
782757,3050167,5658,Petr Cech,2018-08-12,11,2018-05-28,2018-12-19,3000000
...,...,...,...,...,...,...,...,...
40258094,4095368,649317,Jhon Durán,2024-03-17,405,2023-02-04,2023-03-16,12000000
40258125,4095368,649317,Jhon Durán,2024-03-17,405,2023-03-16,2023-06-20,12000000
40258156,4095368,649317,Jhon Durán,2024-03-17,405,2023-06-20,2023-10-09,12000000
40258187,4095368,649317,Jhon Durán,2024-03-17,405,2023-10-09,2023-12-19,17000000


In [479]:
clubs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   club_id                  426 non-null    int64  
 1   club_code                426 non-null    object 
 2   name                     426 non-null    object 
 3   domestic_competition_id  426 non-null    object 
 4   total_market_value       0 non-null      float64
 5   squad_size               426 non-null    int64  
 6   average_age              388 non-null    float64
 7   foreigners_number        426 non-null    int64  
 8   foreigners_percentage    378 non-null    float64
 9   national_team_players    426 non-null    int64  
 10  stadium_name             426 non-null    object 
 11  stadium_seats            426 non-null    int64  
 12  net_transfer_record      426 non-null    object 
 13  coach_name               0 non-null      float64
 14  last_season              4

In [480]:
clubs = clubs.rename(columns={'name' : 'club_name'})

In [481]:
clubs = clubs[['club_id', 'club_name']]

In [482]:
df3 = pd.merge(df2, clubs, left_on='player_club_id', right_on='club_id', how='inner')

In [483]:
df3

,game_id,player_id,name,date,player_club_id,valuations_date,lead_date,market_value_in_eur,club_id,club_name
0,3050172,54906,Matteo Darmian,2018-08-10,985,2016-06-01,2016-08-01,15000000,985,Manchester United Football Club
1,3050172,191614,Fred,2018-08-10,985,2014-12-25,2015-06-07,12000000,985,Manchester United Football Club
2,3050172,191614,Fred,2018-08-10,985,2014-06-30,2014-12-25,12000000,985,Manchester United Football Club
3,3050172,191614,Fred,2018-08-10,985,2013-07-22,2014-06-30,11000000,985,Manchester United Football Club
4,3050172,191614,Fred,2018-08-10,985,2013-07-10,2013-07-22,8000000,985,Manchester United Football Club
...,...,...,...,...,...,...,...,...,...,...
1216647,4095366,77757,Thomas Kaminski,2024-03-16,1031,2023-03-24,2023-06-01,2500000,1031,Luton Town Football Club
1216648,4095366,61842,Andros Townsend,2024-03-16,1031,2014-01-15,2014-08-12,7000000,1031,Luton Town Football Club
1216649,4095366,61842,Andros Townsend,2024-03-16,1031,2012-09-16,2012-11-15,500000,1031,Luton Town Football Club
1216650,4095366,649452,Issa Kaboré,2024-03-16,1031,2021-12-16,2022-05-30,3000000,1031,Luton Town Football Club


In [484]:
df3 = df3.drop(['club_id'], axis=1)

In [485]:
df3 = df3[['game_id','date','player_id','name','player_club_id','club_name','valuations_date','lead_date','market_value_in_eur']]
df3 = df3.sort_values(['game_id', 'player_id', 'valuations_date'])

In [486]:
df3 = df3[df3['lead_date'] >= '2018-01-01']

In [487]:
df3[df3['player_id'] == 429014]

,game_id,date,player_id,name,player_club_id,club_name,valuations_date,lead_date,market_value_in_eur
445459,3050176,2018-08-11,429014,Morgan Gibbs-White,543,Wolverhampton Wanderers Football Club,2018-08-11,2018-08-13,1000000
445568,3050183,2018-08-18,429014,Morgan Gibbs-White,543,Wolverhampton Wanderers Football Club,2018-08-11,2018-08-13,1000000
445567,3050183,2018-08-18,429014,Morgan Gibbs-White,543,Wolverhampton Wanderers Football Club,2018-08-13,2018-12-19,1000000
446110,3050216,2018-09-16,429014,Morgan Gibbs-White,543,Wolverhampton Wanderers Football Club,2018-08-11,2018-08-13,1000000
446107,3050216,2018-09-16,429014,Morgan Gibbs-White,543,Wolverhampton Wanderers Football Club,2018-08-13,2018-12-19,1000000
...,...,...,...,...,...,...,...,...,...
1209487,4095366,2024-03-16,429014,Morgan Gibbs-White,703,Nottingham Forest Football Club,2022-09-15,2022-11-03,25000000
1209438,4095366,2024-03-16,429014,Morgan Gibbs-White,703,Nottingham Forest Football Club,2022-11-03,2023-03-16,25000000
1209374,4095366,2024-03-16,429014,Morgan Gibbs-White,703,Nottingham Forest Football Club,2023-03-16,2023-06-20,30000000
1209361,4095366,2024-03-16,429014,Morgan Gibbs-White,703,Nottingham Forest Football Club,2023-06-20,2023-12-19,40000000


In [511]:
filtered_df

,game_id,date,player_id,name,player_club_id,club_name,valuations_date,lead_date,market_value_in_eur
678261,3050167,2018-08-12,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2018-06-07,2018-12-19,1500000
680641,3050247,2018-10-22,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2018-06-07,2018-12-19,1500000
680829,3050259,2018-10-28,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2018-06-07,2018-12-19,1500000
682188,3056407,2018-12-05,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2018-06-07,2018-12-19,1500000
682517,3050317,2018-12-08,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2018-06-07,2018-12-19,1500000
...,...,...,...,...,...,...,...,...,...
1208524,4095332,2024-02-24,1005649,Murillo,703,Nottingham Forest Football Club,2023-12-19,2024-03-14,22000000
1208846,4095347,2024-03-02,1005649,Murillo,703,Nottingham Forest Football Club,2023-12-19,2024-03-14,22000000
1209066,4095353,2024-03-10,1005649,Murillo,703,Nottingham Forest Football Club,2023-12-19,2024-03-14,22000000
1209421,4095366,2024-03-16,1005649,Murillo,703,Nottingham Forest Football Club,2024-03-14,2024-04-15,25000000


In [488]:
# 'appearances_date' 열을 datetime 형식으로 변환
df3['date'] = pd.to_datetime(df3['date'])

# appearances_date와 valuations_date 사이의 차이 계산
df3['date_diff'] = (df3['valuations_date'] - df3['date']).abs()

# 중복된 행 중에서 appearances_date와 가장 가까운 행 선택
idx = df3.groupby(['player_id', 'player_club_id', 'date'])['date_diff'].idxmin()

# 선택된 인덱스를 제외한 나머지 행 삭제
filtered_df = df3.loc[idx]

# 불필요한 열 삭제
filtered_df = filtered_df.drop(columns=['date_diff'])

In [512]:
grouped_df = filtered_df.groupby(['club_name', 'date', 'game_id', 'player_club_id']).agg({
    'market_value_in_eur': ['sum', 'mean']
}).reset_index()

In [528]:
grouped_df = pd.read_csv('grouped_df_sum_mean.csv')


,club_name,date,game_id,player_club_id,market_value_in_eur_sum,market_value_in_eur_mean
0,Arsenal Football Club,2018-08-12,3050167,11,429000000,30642857.14
1,Arsenal Football Club,2018-08-18,3050180,11,453000000,32357142.86
2,Arsenal Football Club,2018-08-25,3050187,11,423000000,30214285.71
3,Arsenal Football Club,2018-09-02,3050199,11,453000000,32357142.86
4,Arsenal Football Club,2018-09-15,3050212,11,453000000,32357142.86
...,...,...,...,...,...,...
4361,Wolverhampton Wanderers Football Club,2024-02-10,4095319,543,255000000,17000000.00
4362,Wolverhampton Wanderers Football Club,2024-02-17,4095329,543,243000000,17357142.86
4363,Wolverhampton Wanderers Football Club,2024-02-25,4095339,543,244000000,17428571.43
4364,Wolverhampton Wanderers Football Club,2024-03-02,4095346,543,211000000,14066666.67


In [529]:
grouped_df = grouped_df.drop(['Unnamed: 0'], axis=1)

In [530]:
grouped_df

,club_name,date,game_id,player_club_id,market_value_in_eur_sum,market_value_in_eur_mean
0,Arsenal Football Club,2018-08-12,3050167,11,429000000,30642857.14
1,Arsenal Football Club,2018-08-18,3050180,11,453000000,32357142.86
2,Arsenal Football Club,2018-08-25,3050187,11,423000000,30214285.71
3,Arsenal Football Club,2018-09-02,3050199,11,453000000,32357142.86
4,Arsenal Football Club,2018-09-15,3050212,11,453000000,32357142.86
...,...,...,...,...,...,...
4361,Wolverhampton Wanderers Football Club,2024-02-10,4095319,543,255000000,17000000.00
4362,Wolverhampton Wanderers Football Club,2024-02-17,4095329,543,243000000,17357142.86
4363,Wolverhampton Wanderers Football Club,2024-02-25,4095339,543,244000000,17428571.43
4364,Wolverhampton Wanderers Football Club,2024-03-02,4095346,543,211000000,14066666.67


In [517]:
clubs_raw

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
0,105,sv-darmstadt-98,Sportverein Darmstadt 1898 e. V.,L1,NaN,31,26.6,11,35.5,1,Merck-Stadion am Böllenfalltor,17810,€-1.48m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/sv-darmstadt-9...
1,11127,ural-ekaterinburg,FK Ural Yekaterinburg,RU1,NaN,27,27.9,15,55.6,6,Yekaterinburg Arena,23000,€-895k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/ural-ekaterinb...
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,NaN,35,26.7,17,48.6,13,Tüpraş Stadyumu,42590,€-26.00m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/besiktas-istan...
3,12,as-rom,Associazione Sportiva Roma,IT1,NaN,26,26.9,18,69.2,16,Olimpico di Roma,73261,+€63.00m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/as-rom/startse...
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,NaN,26,25.4,20,76.9,21,Tottenham Hotspur Stadium,62850,€-151.40m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/tottenham-hots...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,68608,cf-os-belenenses,CF Os Belenenses,PO1,NaN,30,25.7,5,16.7,0,Estádio do Restelo,19980,+-0,NaN,2017,../data/raw/transfermarkt-scraper/2017/clubs.j...,https://www.transfermarkt.co.uk/cf-os-belenens...
422,724,fc-volendam,Football Club Volendam,NL1,NaN,31,22.2,17,54.8,4,Kras Stadion,7384,+€2.30m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/fc-volendam/st...
423,800,atalanta-bergamo,Atalanta Bergamasca Calcio S.p.a.,IT1,NaN,25,26.6,17,68.0,16,Gewiss Stadium,21747,+€73.82m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/atalanta-berga...
424,979,moreirense-fc,Moreirense Futebol Clube,PO1,NaN,29,25.7,17,58.6,3,Estádio C. J. de Almeida Freitas,6153,€-2.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/moreirense-fc/...


In [540]:
grouped_df_sum_mean = grouped_df.merge(clubs_raw[['filename', 'url' ,'club_id']], left_on='player_club_id', right_on='club_id', how='inner')

In [541]:
grouped_df_sum_mean.to_csv('grouped_df_sum_mean.csv', index=False)

In [494]:
players[players['highest_market_value_in_eur'].isnull()]

,player_id,name,player_code,country_of_birth,country_of_citizenship,date_of_birth,sub_position,position,foot,height_in_cm,current_club_domestic_competition_id,current_club_name,highest_market_value_in_eur
1344,15817,Patrick Deman,patrick-deman,Belgium,Belgium,1968-07-31,Goalkeeper,Goalkeeper,NaN,184.0,BE1,Koninklijke Voetbalclub Kortrijk,NaN
6945,85316,Kilian Falcón,kilian-falcon,Spain,Spain,1992-09-28,Goalkeeper,Goalkeeper,NaN,184.0,ES1,Real Zaragoza,NaN
6948,85326,Yeray Gómez,yeray-gomez,Spain,Spain,1992-06-10,Goalkeeper,Goalkeeper,NaN,192.0,ES1,Unión Deportiva Almería S.A.D.,NaN
7247,90145,Anthony Senni,anthony-senni,NaN,Belgium,1987-11-26,Left Midfield,Midfield,NaN,173.0,BE1,Royal Excel Mouscron (-2022),NaN
7267,90587,Jerome Leblois,jerome-leblois,NaN,Belgium,1974-12-09,NaN,Missing,NaN,NaN,BE1,Royal Excel Mouscron (-2022),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30467,1157420,Victor Udoh,victor-udoh,NaN,Nigeria,2004-10-18,Centre-Forward,Attack,left,NaN,BE1,Royal Antwerp Football Club,NaN
30482,1169601,Thibau Loeman,thibau-loeman,Belgium,Belgium,2006-06-01,Centre-Back,Defender,right,189.0,BE1,Yellow-Red Koninklijke Voetbalclub Mechelen,NaN
30486,1176433,Abdoul Kader Ouattara,abdoul-kader-ouattara,NaN,Burkina Faso,2005-05-26,Centre-Forward,Attack,NaN,NaN,BE1,Cercle Brugge Koninklijke Sportvereniging,NaN
30497,1229924,Adama Sidibeh,adama-sidibeh,NaN,The Gambia,1998-06-25,Centre-Forward,Attack,NaN,NaN,SC1,Saint Johnstone Football Club,NaN


In [495]:
valuations.isnull().sum()

player_id              0
valuations_date        0
market_value_in_eur    0
dtype: int64

In [496]:
filtered_df[filtered_df['player_id'] == 563306]

,game_id,date,player_id,name,player_club_id,club_name,valuations_date,lead_date,market_value_in_eur


In [497]:
filtered_df[filtered_df['game_id'] == 3050563].reset_index()

,index,game_id,date,player_id,name,player_club_id,club_name,valuations_date,lead_date,market_value_in_eur
0,545993,3050563,2019-05-12,41416,James McArthur,873,Crystal Palace Football Club,2018-12-19,2019-06-13,6000000
1,545920,3050563,2019-05-12,46791,Bakary Sako,873,Crystal Palace Football Club,2018-05-28,2019-06-13,5000000
2,545944,3050563,2019-05-12,52119,Patrick van Aanholt,873,Crystal Palace Football Club,2018-12-19,2019-06-13,9000000
3,511721,3050563,2019-05-12,61841,Adam Smith,989,Association Football Club Bournemouth,2018-12-19,2019-06-13,5000000
4,546063,3050563,2019-05-12,61842,Andros Townsend,873,Crystal Palace Football Club,2018-12-19,2019-06-13,15000000
5,546071,3050563,2019-05-12,64399,Vicente Guaita,873,Crystal Palace Football Club,2018-12-19,2019-06-13,5000000
6,546114,3050563,2019-05-12,74300,Luka Milivojevic,873,Crystal Palace Football Club,2019-03-05,2019-06-13,20000000
7,545907,3050563,2019-05-12,78959,Martin Kelly,873,Crystal Palace Football Club,2018-12-19,2019-06-13,3000000
8,511718,3050563,2019-05-12,85177,Nathaniel Clyne,989,Association Football Club Bournemouth,2018-12-19,2019-06-13,15000000
9,511626,3050563,2019-05-12,90836,Steve Cook,989,Association Football Club Bournemouth,2018-12-19,2019-06-13,10000000


In [498]:
filtered_df.groupby(['club_name', 'game_id'])['market_value_in_eur'].sum().reset_index()

,club_name,game_id,market_value_in_eur
0,Arsenal Football Club,3050167,429000000
1,Arsenal Football Club,3050180,453000000
2,Arsenal Football Club,3050187,423000000
3,Arsenal Football Club,3050199,453000000
4,Arsenal Football Club,3050212,453000000
...,...,...,...
4361,Wolverhampton Wanderers Football Club,4095319,255000000
4362,Wolverhampton Wanderers Football Club,4095329,243000000
4363,Wolverhampton Wanderers Football Club,4095339,244000000
4364,Wolverhampton Wanderers Football Club,4095346,211000000


In [499]:
appearances[appearances['player_id'] == 563306]

,game_id,player_id,player_club_id,current_club_id,date,competition_id
1031572,3219099,563306,1123,1123,2020-07-26,GB1


In [500]:
pd.DataFrame(filtered_df.groupby(['game_id', 'player_club_id'])['market_value_in_eur'].count())

market_value_in_eur
game_id player_club_id                     
3050167 11                               14
        281                              14
3050168 603                              14
        989                              14
3050169 873                              14
...                                     ...
4095365 931                              16
4095366 703                              14
        1031                             14
4095368 379                              14
        405                              15

[4366 rows x 1 columns]

In [501]:
clubs_raw = pd.read_csv('C:/Users/parks/OneDrive/바탕 화면/DATASET/Football/clubs.csv')
appearances_raw = pd.read_csv('C:/Users/parks/OneDrive/바탕 화면/DATASET/Football/appearances.csv')

#프리미어리그만 뽑기
PL_appearances = appearances_raw[appearances_raw['competition_id'] == 'GB1']

#필요한 칼럼 추출
PL_appearances = PL_appearances[['game_id', 'player_club_id', 'date']]
clubs_name = clubs_raw[['club_id', 'name']]

#두 데이터프레임 merge
club_merge = pd.merge(clubs_name, PL_appearances, left_on='club_id', right_on='player_club_id', how='inner')

In [502]:
##season칼럼 만들기

# 'date' 열을 datetime 형식으로 변환
club_merge['date'] = pd.to_datetime(club_merge['date'])

# 'season' 칼럼 생성
def get_season(date):
    year = date.year
    if date.month >= 8:
        year += 1
    return str(year - 1)[2:] + str(year)[2:]

club_merge['season'] = club_merge['date'].apply(get_season)

#각 팀 season별 포함 여부 count
season_count = club_merge.groupby('name')['season'].nunique().reset_index()

#카운트 횟수 내림차순
season_count = season_count.sort_values(by='season', ascending=False)

In [503]:
season_count

,name,season
0,Arsenal Football Club,12
7,Chelsea Football Club,12
33,West Ham United Football Club,12
30,Tottenham Hotspur Football Club,12
17,Manchester City Football Club,12
15,Liverpool Football Club,12
9,Everton Football Club,12
18,Manchester United Football Club,12
8,Crystal Palace Football Club,11
20,Newcastle United Football Club,11


In [504]:
club_merge['season'].unique()

array(['1213', '1314', '1415', '1516', '1617', '1718', '1819', '1920',
       '2021', '2122', '2223', '2324'], dtype=object)

In [505]:
club_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123609 entries, 0 to 123608
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   club_id         123609 non-null  int64         
 1   name            123609 non-null  object        
 2   game_id         123609 non-null  int64         
 3   player_club_id  123609 non-null  int64         
 4   date            123609 non-null  datetime64[ns]
 5   season          123609 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 5.7+ MB


In [506]:
appearances

,game_id,player_id,player_club_id,current_club_id,date,competition_id
0,2231978,38004,853,235,2012-07-03,CLQ
1,2233748,79232,8841,2698,2012-07-05,ELQ
2,2234413,42792,6251,465,2012-07-05,ELQ
3,2234418,73333,1274,6646,2012-07-05,ELQ
4,2234421,122011,195,3008,2012-07-05,ELQ
...,...,...,...,...,...,...
1556964,4300788,632349,31,31,2024-03-17,FAC
1556965,4300788,69633,985,985,2024-03-17,FAC
1556966,4300788,712117,31,31,2024-03-17,FAC
1556967,4300788,811779,985,985,2024-03-17,FAC


In [507]:
pd.DataFrame(appearances.groupby(['game_id', 'player_club_id'])['player_id'].count())

player_id
game_id player_club_id           
2211607 383                    13
        610                    14
2218677 506                    14
        6195                   14
2219794 58                     14
...                           ...
4300787 281                    13
        762                    16
4300788 31                     16
        985                    16
4300789 543                    13

[116159 rows x 1 columns]

In [508]:
df3[df3['player_id'] == 2865].sort_values('lead_date')

,game_id,date,player_id,name,player_club_id,club_name,valuations_date,lead_date,market_value_in_eur,date_diff
678301,3050167,2018-08-12,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2017-12-29,2018-06-07,2000000,226 days
684679,3050387,2019-01-29,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2017-12-29,2018-06-07,2000000,396 days
683463,3050417,2018-12-26,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2017-12-29,2018-06-07,2000000,362 days
683745,3050352,2018-12-29,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2017-12-29,2018-06-07,2000000,365 days
685657,3050427,2019-02-24,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2017-12-29,2018-06-07,2000000,422 days
683071,3050337,2018-12-22,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2017-12-29,2018-06-07,2000000,358 days
682179,3056407,2018-12-05,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2017-12-29,2018-06-07,2000000,341 days
682672,3050333,2018-12-16,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2017-12-29,2018-06-07,2000000,352 days
685028,3050404,2019-02-03,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2017-12-29,2018-06-07,2000000,401 days
682612,3050317,2018-12-08,2865,Stephan Lichtsteiner,11,Arsenal Football Club,2017-12-29,2018-06-07,2000000,344 days
